In [ ]:
import os
import pandas as pd
import pickle
from typing import List, Dict, Any, Optional
from langchain_core.documents import Document
import warnings
from pathlib import Path
from pprint import pprint

def safe_load_pickle(file_path: str) -> Any:
    """Safely load pickle files that may contain pandas objects"""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            # First try standard pickle load
            with open(file_path, "rb") as f:
                return pickle.load(f)
        except (TypeError, AttributeError, pickle.UnpicklingError):
            try:
                # Fallback to pandas read_pickle
                return pd.read_pickle(file_path)
            except Exception as e:
                raise ValueError(f"Failed to load {file_path}: {str(e)}")

def inspect_file(file_path: str) -> None:
    """Debug function to examine file structure"""
    print(f"\n=== Inspecting {file_path} ===")
    try:
        if file_path.endswith(('.xlsx', '.xls')):
            df = pd.read_excel(file_path)
            print("Excel file detected")
            print(f"Shape: {df.shape}")
            print("Columns:", df.columns.tolist())
            print("\nFirst row:")
            pprint(df.iloc[0].to_dict())
        elif file_path.endswith('.pkl'):
            data = safe_load_pickle(file_path)
            print(f"Type: {type(data)}")
            if isinstance(data, pd.DataFrame):
                print("Pandas DataFrame detected")
                print(f"Shape: {data.shape}")
                print("Columns:", data.columns.tolist())
                print("\nFirst row:")
                pprint(data.iloc[0].to_dict())
            elif isinstance(data, list):
                print(f"List of {len(data)} items")
                if data:
                    print("\nFirst item type:", type(data[0]))
                    if isinstance(data[0], dict):
                        print("Keys in first item:", data[0].keys())
            elif isinstance(data, dict):
                print("Dictionary with keys:", data.keys())
            else:
                print("Content sample:", str(data)[:200] + "...")
        else:
            raise ValueError("Unsupported file format")
    except Exception as e:
        print(f"Inspection failed: {str(e)}")

def load_buffet_qna_xlsx(file_path: str) -> List[Document]:
    """
    Load Q&A data from Excel with columns: Section, Questions, Answers
    Returns List[Document] where:
    - page_content contains formatted Q&A
    - metadata contains structured fields
    """
    df = pd.read_excel(file_path)
    
    # Validate required columns
    required_cols = {'Section', 'Questions', 'Answers'}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns in Q&A file: {missing}")

    documents = []
    for _, row in df.iterrows():
        # Create human-readable content
        page_content = (
            f"Question: {row['Questions']}\n"
            f"Answer: {row['Answers']}\n"
            f"Section: {row['Section']}"
        )
        
        # Store structured data in metadata
        metadata = {
            "section": row["Section"],
            "question": row["Questions"],
            "answer": row["Answers"],
            "source": "buffet_qna"
        }
        
        documents.append(Document(
            page_content=page_content,
            metadata=metadata
        ))
    
    return documents

def load_brka_trades_xlsx(file_path: str) -> List[Document]:
    """
    Load trades data from Excel with financial columns
    Returns List[Document] where:
    - page_content contains key trade info
    - metadata contains all raw data
    """
    df = pd.read_excel(file_path)
    
    # Validate required columns
    required_cols = {'RIC', 'Security Name', 'Date', 'Position', 'Position Change'}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns in trades file: {missing}")

    documents = []
    for _, row in df.iterrows():
        # Create human-readable summary
        page_content = (
            f"Security: {row['Security Name']} ({row['RIC']})\n"
            f"Date: {row['Date']}\n"
            f"Position: {row['Position']:,} shares\n"
            f"Change: {row['Position Change']:+,}"
        )
        
        # Store all raw data in metadata
        metadata = row.to_dict()
        metadata.update({"source": "brka_trades"})
        
        documents.append(Document(
            page_content=page_content,
            metadata=metadata
        ))
    
    return documents

def load_news_pickle(file_path: str) -> List[Document]:
    """Load news pickle with flexible format handling"""
    data = safe_load_pickle(file_path)
    
    # Handle DataFrame case
    if isinstance(data, pd.DataFrame):
        data = data.to_dict('records')
    
    # Handle list of dicts
    if isinstance(data, list) and all(isinstance(x, dict) for x in data):
        return [
            Document(
                page_content=item.get('text', item.get('content', str(item))),
                metadata={k: v for k, v in item.items() 
                         if k not in ['text', 'content']}
            )
            for item in data
        ]
    
    # Handle single dictionary
    elif isinstance(data, dict):
        return [Document(
            page_content=data.get('text', data.get('content', str(data))),
            metadata={k: v for k, v in data.items() 
                     if k not in ['text', 'content']}
        )]
    
    # Fallback for other formats
    return [Document(page_content=str(data))]

def load_shareholder_letters_pickle(file_path: str) -> List[Document]:
    """Load shareholder letters with year-based structure"""
    data = safe_load_pickle(file_path)
    
    if isinstance(data, dict) and all(isinstance(k, (str, int)) for k in data):
        return [
            Document(
                page_content=content,
                metadata={"year": year, "source": "shareholder_letter"}
            )
            for year, content in data.items()
        ]
    
    # Fallback for other formats
    return load_news_pickle(file_path)

def load_documents(file_path: str) -> List[Document]:
    """Main document loading interface"""
    try:
        if file_path.endswith('.xlsx'):
            if 'buffet_qna' in file_path.lower():
                return load_buffet_qna_xlsx(file_path)
            elif 'brka_trades' in file_path.lower():
                return load_brka_trades_xlsx(file_path)
        
        elif file_path.endswith('.pkl'):
            if 'news' in file_path.lower():
                return load_news_pickle(file_path)
            elif 'shareholder' in file_path.lower():
                return load_shareholder_letters_pickle(file_path)
        
        raise ValueError(f"Unrecognized file type: {file_path}")
    
    except Exception as e:
        print(f"Error loading {file_path}: {str(e)}")
        return [Document(
            page_content=f"Error loading document: {str(e)}",
            metadata={"source": "error", "file_path": file_path}
        )]

def load_all_documents(
    base_path: str = "/data",
    buffet_qna_path: Optional[str] = None,
    brka_trades_path: Optional[str] = None,
    news_path: Optional[str] = None,
    shareholder_letters_path: Optional[str] = None,
    verbose: bool = True
) -> List[Document]:
    """
    Load all 4 document sources at once.
    
    Args:
        base_path: Base directory for files if individual paths not specified
        *_path: Override individual file paths
        verbose: Print loading progress
    
    Returns:
        Combined list of Documents from all sources
    """
    # Set default paths if not specified
    buffet_qna_path = buffet_qna_path or Path(base_path) / "buffet_qna.xlsx"
    brka_trades_path = brka_trades_path or Path(base_path) / "brka_trades.xlsx"
    news_path = news_path or Path(base_path) / "news.pkl"
    shareholder_letters_path = shareholder_letters_path or Path(base_path) / "shareholder_letters.pkl"
    
    all_docs = []
    
    # Load each file with progress reporting
    for file_path, loader in [
        (buffet_qna_path, load_buffet_qna_xlsx),
        (brka_trades_path, load_brka_trades_xlsx),
        (news_path, load_news_pickle),
        (shareholder_letters_path, load_shareholder_letters_pickle)
    ]:
        try:
            if verbose:
                print(f"Loading {file_path}...")
            docs = loader(file_path)
            all_docs.extend(docs)
            if verbose:
                print(f"Loaded {len(docs)} documents")
        except Exception as e:
            if verbose:
                print(f"Error loading {file_path}: {str(e)}")
            all_docs.append(Document(
                page_content=f"Error loading {file_path}: {str(e)}",
                metadata={"source": "error", "file_path": str(file_path)}
            ))
    
    if verbose:
        print(f"\nTotal documents loaded: {len(all_docs)}")
    
    return all_docs

# print(f"Script running from: {os.getcwd()}")

# print(load_documents("../data/news.pkl"))
# print(load_all_documents("../data"))

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


CHUNK_SIZE = 1024
CHUNK_OVERLAP = 20

def split_documents(documents):
    """
    Splits a list of document objects into manageable text chunks.
    Assumes each document has a 'content' field.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    # For compatibility with langchain's splitter, wrap your dicts in a simple object:
    # Here we assume each document is a dict with a 'content' key.
    # You might need to convert these dicts to the Document type expected by langchain.
    return splitter.split_documents(documents)


In [ ]:
import numpy as np
from rank_bm25 import BM25Okapi
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import EnsembleRetriever

# Compare 2 different retrieval methods and evaluate

def build_bm25(corpus):
    """
    Builds a BM25 model from a list of texts (corpus).
    """
    return BM25Okapi(corpus)

def build_faiss_index(documents):
    """
    Uses OpenAI embeddings to build a FAISS vectorstore from documents.
    """
    embeddings = OpenAIEmbeddings()
    # FAISS vectorstore will build the index based on document 'content'
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore

def create_ensemble_retriever(bm25_retriever, faiss_retriever):
    """
    Combines BM25 and FAISS retrievers into an ensemble retriever.
    """
    return EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever])


In [ ]:
import numpy as np
from sentence_transformers import CrossEncoder
from typing import List, Dict, Any, Optional
from langchain_core.documents import Document



# Initialize the pre-trained cross-encoder (adjust model name as needed)
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def rerank_documents(query: str, documents: List[Document], top_k: int = 5) -> List[Document]:
    """
    Reranks documents using a cross-encoder model.
    
    Args:
        query: The search query
        documents: List of LangChain Document objects to rerank
        top_k: Number of top documents to return
        
    Returns:
        List of reranked Document objects
    """
    # Create pairs of (query, document_content)
    pairs = [[query, doc.page_content] for doc in documents]  # Changed to use page_content
    
    # Get scores from cross-encoder
    scores = cross_encoder.predict(pairs)
    
    # Sort by scores in descending order and get top_k indices
    ranked_indices = np.argsort(scores)[::-1][:top_k]
    
    # Return documents in ranked order
    return [documents[i] for i in ranked_indices]

In [ ]:
from typing import List, Dict, Any, Optional
from langchain_core.documents import Document

def create_prompt(query: str, company_name: str, documents: List[Document]) -> str:
    """
    Creates a focused prompt for the LLM using both the query and retrieved documents.
    
    Args:
        query: The original user query/search terms
        company_name: Target company for the report
        documents: List of retrieved LangChain Document objects
        
    Returns:
        Formatted prompt string
    """
    context = "\n".join([f"Source {i+1}:\n{doc.page_content}\n" 
                        for i, doc in enumerate(documents)])
    
    prompt = f"""
**Task**: Generate a comprehensive investment report for {company_name} that specifically addresses: 
"{query}"

**Context from company documents**:
{context}

**Report Requirements**:
1. Directly respond to the query about {query}
2. Focus on these key aspects (if relevant):
   - Revenue growth trends and drivers
   - Profitability metrics and margins
   - Strategic initiatives and investments
   - Competitive positioning
   - Risks and challenges
3. Include specific numbers, percentages, and timeframes when available
4. Highlight any contradictions or uncertainties in the data
5. Maintain objective, professional tone

**Report Structure**:
[Overview] - Brief introduction addressing the query
[Key Findings] - Bullet points of most relevant insights
[Detailed Analysis] - Expanded discussion with supporting data
[Conclusions] - Summary and forward-looking statements

**Begin Report for {company_name}**:
"""
    return prompt.strip()

def create_evaluation_prompt(query: str, generated_report: str, 
                           retrieved_documents: List[Document]) -> str:
    """
    Creates an evaluation prompt that assesses how well the report addresses the query.
    
    Args:
        query: Original user question/search terms
        generated_report: LLM-generated report to evaluate
        retrieved_documents: Documents used for context
        
    Returns:
        Formatted evaluation prompt
    """
    context_samples = "\n".join([f"• {doc.page_content[:200]}..." 
                               for doc in retrieved_documents[:3]])
    
    prompt = f"""
**Evaluation Task**: Assess how well this investment report addresses the original query.

**Original Query**: "{query}"

**Generated Report**:
{generated_report}

**Sample Supporting Context**:
{context_samples}

**Evaluation Rubric**:
1. Query Relevance (0-10): 
   - Does every section directly address aspects of "{query}"?
   - Are there any irrelevant tangents?

2. Data Accuracy (0-10):
   - Are all facts supported by the context documents?
   - Are numbers and claims properly qualified?

3. Analytical Depth (0-10):
   - Does it surface non-obvious insights from the data?
   - Does it identify relationships between different metrics?

4. Actionability (0-10):
   - Would an investor find clear takeaways?
   - Are risks and opportunities properly highlighted?

**Output Format**:
- For each criterion: 
  [Score] [Justification in 1-2 sentences]
- Overall summary (1 paragraph)

**Begin Evaluation**:
"""
    return prompt.strip()

In [ ]:
import os
from dotenv import load_dotenv
import openai 
from typing import List, Dict, Any, Optional, Callable
from langchain_core.documents import Document
from rag.text_splitter import split_documents
from rag.retrieval import build_bm25, build_faiss_index, create_ensemble_retriever
from rag.reranker import rerank_documents
from rag.prompt_engineering import create_prompt, create_evaluation_prompt

from langchain_community.retrievers import BM25Retriever

# load env variables from .env
load_dotenv()

# set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise ValueError("API key is not set")

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

class RAGPipeline:
    def __init__(
        self, 
        raw_documents,
        llm_type: str = "openai",  # "openai" or "custom"
        custom_llm: Optional[Callable] = None,
        openai_model: str = "gpt-3.5-turbo"
    ):
        """
        Initialize RAG pipeline with LLM options.
        
        Args:
            raw_documents: Input documents for retrieval
            llm_type: "openai" or "custom"
            custom_llm: Function(prompt: str, context: str) -> str
            openai_model: OpenAI model name if using OpenAI
        """
        self.documents = split_documents(raw_documents)
        self.context_memory = []
        self.llm_type = llm_type
        self.custom_llm = custom_llm
        self.openai_model = openai_model
        
        if llm_type == "openai":
            import openai
            openai.api_key = os.getenv("OPENAI_API_KEY")
            if not openai.api_key:
                raise ValueError("OPENAI_API_KEY not found in .env file")
        
        self._initialize_retrievers()

    def _initialize_retrievers(self):
        """Initialize BM25 and FAISS retrievers"""
        # BM25 Retriever
        from langchain.retrievers import BM25Retriever
        self.bm25_retriever = BM25Retriever.from_documents(self.documents)
        self.bm25_retriever.k = 5
        
        # FAISS Retriever
        self.faiss_retriever = build_faiss_index(self.documents).as_retriever(search_kwargs={"k": 5})
        
        # Ensemble Retriever
        self.ensemble_retriever = create_ensemble_retriever(
            self.bm25_retriever, 
            self.faiss_retriever
        )
    
    def generate_text(self, prompt: str, context: str = "") -> str:
        """
        Unified text generation interface.
        Routes to OpenAI or custom LLM based on configuration.
        """
        if self.llm_type == "custom" and self.custom_llm:
            return self.custom_llm(prompt, context)
        elif self.llm_type == "openai":
            return self._call_openai(prompt, context)
        else:
            raise ValueError("Invalid LLM configuration")

    def _call_openai(self, prompt: str, context: str = "") -> str:
        """Internal method for OpenAI API calls"""
        try:
            import openai
            messages = []
            if context:
                messages.append({"role": "system", "content": context})
            messages.append({"role": "user", "content": prompt})
            
            response = openai.ChatCompletion.create(
                model=self.openai_model,
                messages=messages,
                temperature=0.7,
                max_tokens=1500
            )
            return response.choices[0].message['content']
        except Exception as e:
            print(f"Error calling OpenAI: {e}")
            return f"Error generating response: {str(e)}"

    def retrieve_and_rerank(self, query: str, retriever, top_k: int = 5) -> List[Document]:
        """
        Retrieves and reranks documents.
        """
        docs = retriever.get_relevant_documents(query)
        return rerank_documents(query, docs, top_k=top_k)

    def generate_report(self, query: str, company_name: str, retriever) -> tuple:
        """
        Generates a report using retrieved documents.
        """
        retrieved_docs = self.retrieve_and_rerank(query, retriever)
        prompt = create_prompt(query, company_name, retrieved_docs)
        context_text = "\n".join(self.context_memory)
        
        report = self.generate_text(prompt, context=context_text)
        
        # Update context memory
        self.context_memory.append(f"Query: {query}")
        self.context_memory.append(f"Report: {report[:200]}...")
        
        return report, retrieved_docs

    def evaluate_report(self, query: str, report: str, retrieved_docs: List[Document]) -> str:
        """
        Evaluates the generated report.
        """
        eval_prompt = create_evaluation_prompt(query, report, retrieved_docs)
        return self.generate_text(eval_prompt)

    def process_query(self, query: str, company_name: str, method: str = "ensemble") -> tuple:
        """
        Main method to process a query through the RAG pipeline.
        """
        retriever = {
            "bm25": self.bm25_retriever,
            "faiss": self.faiss_retriever,
            "ensemble": self.ensemble_retriever
        }.get(method, self.ensemble_retriever)
        
        report, retrieved_docs = self.generate_report(query, company_name, retriever)
        evaluation = self.evaluate_report(query, report, retrieved_docs)
        
        return report, evaluation

In [ ]:
# Document loading
if args.load_all:
    print("Loading all standard documents from data/ directory...")
    raw_docs = load_all_documents(base_path="../data")
else:
    raw_docs = load_documents(args.data_path)

# Initialize and run pipeline
pipeline = RAGPipeline(raw_docs)
report, evaluation = pipeline.process_query(
    args.query, args.company, method=args.method
)
